<a href="https://colab.research.google.com/github/Milad-Khanchi/Deep_QLearning/blob/main/Copy_of_Deep_Q_Learning_for_Lunar_Landing_Partial_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,280 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../swi

### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [135]:
class Network(nn.Module):
    def __init__(self, input_size, output_size, seed=42):
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [136]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # The Lunar Lander environment was upgraded to v3
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [137]:
lr = 5e-4
b_size = 100
gamma = 0.99
buffer_size = int(1e5)
tau = 1e-3

In [138]:
model = Network(state_size, number_actions)

### Implementing Experience Replay

In [186]:
class ReplayMemory(object):
    def __init__(self, capacity):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []

    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]

    def sample(self, batch_size):
    # Unzip the sampled experiences
        samples = random.sample(self.memory, batch_size)
        states, actions, rewards, next_states, dones = zip(*samples)

    # Convert to torch tensors with consistent shapes
        states = torch.tensor(np.array(states)).float().to(self.device)  # Ensure 2D shape: (batch_size, state_size)
        actions = torch.tensor(np.array(actions)).long().unsqueeze(1).to(self.device)  # Ensure 2D shape: (batch_size, 1)
        rewards = torch.tensor(np.array(rewards)).float().unsqueeze(1).to(self.device)  # Ensure 2D shape: (batch_size, 1)
        next_states = torch.tensor(np.array(next_states)).float().to(self.device)  # Ensure 2D shape: (batch_size, state_size)
        dones = torch.tensor(np.array(dones).astype(np.uint8)).float().unsqueeze(1).to(self.device)  # Ensure 2D shape: (batch_size, 1)

        return states, actions, rewards, next_states, dones


### Implementing the DQN class

In [187]:
class Agent():
    def __init__(self, state_size, action_size):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.state_size = state_size
        self.action_size = action_size
        self.local_network = Network(state_size, action_size).to(self.device)
        self.target_network = Network(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_network.parameters(), lr=lr)
        self.memory = ReplayMemory(buffer_size)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        self.memory.push((state, action, reward, next_state, done))
        self.t_step = (self.t_step + 1) % 4
        if self.t_step == 0:
            if len(self.memory.memory) > b_size:
                experiences = self.memory.sample(b_size)
                self.learn(experiences, gamma)

    def act(self, state, eps=0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.local_network.eval()
        with torch.no_grad():
            action_values = self.local_network(state)
        self.local_network.train()
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences
        next_Q_targets = self.target_network(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (gamma * next_Q_targets * (1 - dones))
        Q_expected = self.local_network(states).gather(1, actions)
        loss = F.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.soft_update(self.local_network, self.target_network, tau)

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

### Initializing the DQN agent

In [188]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [189]:
number_episodes = 2000
max_t = 1000
eps_start = 1.0
eps_end = 0.01
eps_decay = 0.995
epsilon = eps_start
scores = deque(maxlen=100)

for i_episode in range(1, number_episodes+1):
    state, _ = env.reset()
    score = 0
    for t in range(max_t):
        action = agent.act(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)

        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break
    scores.append(score)
    epsilon = max(eps_end, eps_decay*epsilon)
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores)), end="")
    if i_episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores)))
        if np.mean(scores)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores)))
            torch.save(agent.local_network.state_dict(), 'checkpoint.pth')
            break

Episode 100	Average Score: -181.53
Episode 200	Average Score: -113.20
Episode 300	Average Score: -40.43
Episode 400	Average Score: 12.54
Episode 500	Average Score: 114.93
Episode 600	Average Score: 133.20
Episode 700	Average Score: 183.21
Episode 800	Average Score: 217.47

Environment solved in 700 episodes!	Average Score: 217.47


## Part 3 - Visualizing the results

In [191]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()